In [1]:
import pennylane as qml
import tensorflow as tf

In [2]:
num_of_qubits = 4

In [3]:
def data_encoding(inputs):
    for index, data_input in enumerate(inputs):
        qml.RX(data_input, wires=index)

def quantum_black_box(weights, n_qubits):
    for idx in range(n_qubits-1):
        qml.CNOT(wires=[idx, idx+1])
    for idx in range(n_qubits): 
        qml.RY(weights[idx], wires=idx)

In [4]:
dev = qml.device('default.qubit', wires=num_of_qubits, shots=1000)
#print(f'Device (dev):\n----------------------------------\n{dev}')

@qml.qnode(dev)
def qnode(inputs, weights):
    #print('Inputs  :', inputs)
    #print('Weights :', weights, '\n')
    
    data_encoding(inputs)
    quantum_black_box(weights, num_of_qubits)

    return tuple([qml.expval(qml.PauliZ(i)) for i in range(num_of_qubits)])

weight_shapes = {"weights": 4}

In [5]:
inp = [0, 0.1, 1.5, 0.33]
wts = [0.25, 0.33, 0.75, 0.67]

In [6]:
print(qml.draw(qnode)(inputs=inp, weights=wts))

0: ──RX(0.00)─╭●──RY(0.25)─────────────────────┤  <Z>
1: ──RX(0.10)─╰X─╭●─────────RY(0.33)───────────┤  <Z>
2: ──RX(1.50)────╰X────────╭●─────────RY(0.75)─┤  <Z>
3: ──RX(0.33)──────────────╰X─────────RY(0.67)─┤  <Z>


In [7]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=4, dtype='float64')

In [8]:
qlayer.get_config()

{'name': 'keras_layer',
 'trainable': True,
 'dtype': 'float64',
 'dynamic': True,
 'output_dim': 4,
 'weight_specs': {},
 'weight_shapes': {'weights': (4,)},
 'argnum': None}

In [9]:
t = qlayer.qnode(inp, wts)
print(t)

tf.Tensor([0.966 0.94  0.036 0.028], shape=(4,), dtype=float64)
